In [3]:
import requests
from ast import literal_eval
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed

In [4]:
# NOT USED FOR NOW
# poloniex_market_pairs = {}
# with open('poloniex_market_id.txt') as f:
#     # get the currency pair ID's for use in poloniex extraction
#     for line in f:
#         num_id, pair = line.split()
#         poloniex_market_pairs[pair] = int(num_id)

In [5]:
pol_fees = {'maker': 0.15, 'taker': 0.25, 'btc_withdraw': 0.0005}  # 'maker' and 'taker' reflect exchange fee
to_fees = {'maker': 0.2, 'taker': 0.2, 'btc_withdraw': 0.00005}  # 'btc_withdraw' is flat BTC withdraw fee

In [6]:
def get_data_tradeogre(coin='GRIN'):  # https://tradeogre.com/help/api
    """
    Gets market data from TradeOgre API for specified coin and returns the dictionary.
    Adds the columns 'taker' and 'maker,' which is the bid and ask prices for the coin, respectively.
    This is because a taker (selling) transaction would fill at the highest bid, while a maker (selling) transaction
    would fill at (up to) the lowest ask.
    
    Parameters
    ----------
    
    coin : string
            Specifies symbol of coin to search for. e.g. "GRIN, ETC"
        
    """
    page = requests.get('https://tradeogre.com/api/v1/markets')
    page.raise_for_status()
    for d in literal_eval(page.text):
        try:
            dic = d[f'BTC-{coin}']
            # adds these keys for use in sell_amount later
            dic['taker'] = float(dic['bid'])  # taker sale fills at highest bid
            dic['maker'] = float(dic['ask'])  # maker sale fills at (up to) lowest ask
            return dic
        except KeyError:
            pass
    return None

def get_data_poloniex(coin='GRIN'):  # https://docs.poloniex.com/#introduction
    """
    Gets market data from Poloniex API for specified coin and returns the dictionary. Near identical functionality
    to get_data_tradeogre (see for further documentation), but with slight modifications due to TradeOgre 
    and Poloniex APIs returning data in different formats.
    """
    
    page = requests.get('https://poloniex.com/public?command=returnTicker')
    page.raise_for_status()
    data = literal_eval(page.text)
    
    try:
        dic = data[f'BTC_{coin}']
        dic['taker'] = float(dic['highestBid'])  # adds these keys for use in sell_amount later
        dic['maker'] = float(dic['lowestAsk'])
        return dic
    except KeyError:
        return None

def get_btc_price():
    """
    Gets current btc price from https://www.coindesk.com/api as a float.
    """
    
    page = requests.get('https://api.coindesk.com/v1/bpi/currentprice.json')
    data = literal_eval(page.text)
    return float(data['bpi']['USD']['rate'].replace(',',""))

def total_profit(rate, amount, trade_fee=0, withdraw_fee=0):
    """
    Returns total BTC received, given trading fee (maker/taker fee, in %), withdraw fee (in BTC), and exchange rate.
    Simply uses formula (rate*amount*(1-trade_fee/100))-withdraw_fee.
    
    Parameters
    ----------
    
    rate : float
            Exchange rate of the desired coin, i.e. the price of one coin in BTC.
    amount : float
            Amount of coins being traded for BTC.
    trade_fee : float
            Fee charged by the exchange for fulfillment of the transaction, as a percent of the transaction value.
    withdraw_fee : float
            Flat fee charged when withdrawing BTC to an external wallet.       
    """
    trade_fee /= 100
    return (rate*amount*(1-trade_fee))-withdraw_fee

def sell_amount(tx_type='taker', amount=0, to=None, pol=None, btc_price = 0):
    """
    Manipulatable function taking advantage of total_profit and widgets to print formatted data.
    to and pol parameters should be the dictionaries from get_data_[exchange].
    
    Parameters
    ----------
    
    tx_type : {'taker','maker'}
            One of two order types, for use with popular taker/maker fee structuring. Used to calculate rate for
            total_profit.
    amount : float
            Amount of coins being traded for BTC. Passed directly into total_profit.
    to : dict or None
            Dictionary returned from get_data_tradeogre function. Specifying None value here will give 'N/A' when
            results are displayed.
    pol : dict or None
            Dictionary returned from get_data_poloniex function. Same behavior as to when specifying None value.
    btc_price : float
            Current/desired price of BTC to use for USD conversion.
    
    """
    if to:  # to = None occurs when coin data couldn't be pulled from API
        to_rate = to[tx_type]
        to_fee = to_fees[tx_type]
        to_sellamt = total_profit(to_rate, amount, to_fee, to_fees['btc_withdraw'])
        to_str = f'{to_sellamt:.7f} (${btc_price*to_sellamt:.2f})'
    else:
        to_str = 'N/A'
        
    if pol:
        pol_rate = pol[tx_type]
        pol_fee = pol_fees[tx_type]
        pol_sellamt = total_profit(pol_rate, amount, pol_fee, pol_fees['btc_withdraw'])
        pol_str = f'{pol_sellamt:.7f} (${btc_price*pol_sellamt:.2f})'
    else:
        pol_str = 'N/A'
    
    if not (to or pol):  # no coin data on either exchange
        output = widgets.HTML(value="<font size = 3><font color = 'red'>No data found.")
    elif to and ((not pol) or (to_sellamt > pol_sellamt)):  # either only tradeogre data exists or tradeogre rev > pol rev
        output = widgets.HTML(value=f"<font size = 3><font color = 'green'>"
                                    f"TradeOgre Revenue: {to_str}</font>"
                                    f"<br><font color = 'red'>"
                                    f"Poloniex Revenue: {pol_str}")
    else:  # only pol exists or pol rev > to rev
        output = widgets.HTML(value=f"<font size = 3><font color = 'red'>"
                                    f"TradeOgre Revenue: {to_str}</font>"
                                    f"<br><font color = 'green'>"
                                    f"Poloniex Revenue: {pol_str}")
    display(output)
    
def print_data(coin='GRIN', pol=None, to=None):
    """
    Pulls data from exchange APIs and prints out data about spread, ask, and bid prices. Also includes an ipywidgets
    interact call to sell_amount for the revenue calculator. Built for use with ipywidgets interact itself, for
    easy switching between desired coins.
    
    Parameters
    ----------
    
    coin : string
            Specifies symbol of coin to search for.
    """
    print(f'BTC Price: ${get_btc_price()}\n')
    
    if pol:
        print(
        f'Poloniex {coin}-BTC\n'
        f'{"Spread:":>7} {(float(pol["lowestAsk"])-float(pol["highestBid"]))/float(pol["last"])*100:.2f}%\n'
        f'{"Ask:":>7} {pol["lowestAsk"]}\n'
        f'{"Bid:":>7} {pol["highestBid"]}\n')
    else:
        print(f'Could not get {coin} data from Poloniex.\n\n')
    if to:
        print(
        f'Trade Ogre {coin}-BTC\n'
        f'{"Spread:":>7} {(float(to["ask"])-float(to["bid"]))/float(to["price"])*100:.2f}%\n'
        f'{"Ask:":>7} {to["ask"]}\n'
        f'{"Bid:":>7} {to["bid"]}')
    else:
        print(f'Could not get {coin} data from TradeOgre.\n\n')

            
def compare(coin='GRIN'):
    """
    Manipulatable function combining the features of print_data and sell_amount.
    """
    pol = get_data_poloniex(coin)
    to = get_data_tradeogre(coin)
            
    d = interact(print_data, coin=fixed(coin), to=fixed(to), pol=fixed(pol))
    c = interact(sell_amount, tx_type=['taker','maker'],
                    amount=widgets.BoundedFloatText(value=0,min=0,max=2**32,step=0.1,description=f'amt {coin}:'),
                    to=fixed(to), pol=fixed(pol), btc_price=fixed(get_btc_price()))
    

In [7]:
interact(compare, coin=['GRIN','BEAM','ETH','RVN']);

interactive(children=(Dropdown(description='coin', options=('GRIN', 'BEAM', 'ETH', 'RVN'), value='GRIN'), Outp…